### 0. 事前準備

以下のデータを取得して、このノートブックと同じディレクトリにある`data`ディレクトリに配置します。

__UNESCO Syndication__  
  URL:  
  ・https://whc.unesco.org/en/syndication    
  DATA:  
  ・whc-sites-2021.xls  
  データの取得方法:  
    「The List」にある「XLS」リンクをクリックします。

In [ ]:
%matplotlib inline

from bs4 import BeautifulSoup
import folium
from IPython.core.display import HTML
from IPython.display import Image, display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import requests
import seaborn as sns
from selenium import webdriver

### 1. データの概要を掴む

世界遺産データをロード。

データの中身を確認。

各列の状況を`info()`で確認。

各列が何を意味しているか分かりますか？

参考）  
・UNESCO The Criteria for Selection  
　http://whc.unesco.org/en/criteria/

・Template:UNESCO World Heritage Site row/doc  
　https://en.wikipedia.org/wiki/Template:UNESCO_World_Heritage_Site_row/doc

どんな世界遺産があるのかUNESCOのサイトで確認してみましょう。  

`states_name_en`列にある任意の国名で絞って`id_no`を取得し、ギャラリーへのリンクのリストを作成してみます。  
URLのパターン: `https://whc.unesco.org/en/list/<id_noを指定>/gallery/`

まず`states_name_en`にどんな国名があるか`unique()`で確認します。

複数の国に跨っているケースがあって、その場合はカンマ区切りになっているようなので、それぞれを独立した行にしてしまいます。

手順は以下です。  
1.`states_name_en`列の値を行ごとにカンマで分割してリストにし、新たな列`single_country`に格納。  
2.`single_country`列に`explode()`を実行し、リストの要素ごとの行を作成する。

`single_country`の値を`unique()`で確認します。

ギャラリーURLのリストを作成します。

### 2. groupbyによる集約

ここでは、`groupby()`を利用してデータを集約してみます。

#### 2-1. 簡単な集計関数の利用

国別の世界遺産数を取得し、多い順にソートします。

`groupby()`で`DataFrameGroupBy`オブジェクトを生成し、`size()`で国ごとの件数を計算します。

横棒グラフですべての国の世界遺産数を多い順に表示してみましょう。

どんなことが読み取れますか？

同様に地域でグループ化し、多い順に横棒グラフで表示してみます。

複数の国に跨っているものを複数の行にさせたので`id_no`の重複がありますから、`nunique()`を使って件数を取得します。

グラフを表示します。

どんなことが読み取れますか？

#### 2-2. agg関数の利用

複数の任意の集約関数を利用したい場合、`agg()`関数を使って集約関数をまとめて指定できます。

世界遺産のヘクタール面積の最大値、最小値、平均、中央値を`agg()`を使って取得してみましょう。

もちろんこの場合は`agg()`を使わずに`describe()`で簡単に計算できます。

#### 2-3. apply関数の利用

任意の関数を集約単位で実行したい場合、`apply()`を利用します。

地域ごと、国ごとの危機遺産率を取得してみましょう。

最初に、引数として遺産データフレームを取って危機遺産率を計算する関数を定義します。

関数をデータ全体に対して実行してみましょう。

この関数を`apply()`に引数として渡すことで、集約単位ごとに実行することができます。

地域ごとの危機遺産率を取得し、降順で表示してみましょう。

どんなことが読み取れますか？

同様に国ごとの危機遺産率を取得し、降順で表示してみましょう。

国ごとの危機遺産率を以下のルールで地図上に表示します。

- `plotly`を使って地球儀を表示。
- 危機遺産率が0でないものが対象。
- 国の位置を赤い点で示し、危険遺産率に合わせて色を変化させる。
- 国の位置座標は大雑把に国ごとの世界遺産の緯度、経度の平均値とする。

まず世界遺産のデータフレームを国でグループ化し、緯度、経度の平均を計算したデータフレームを作成します。

作成したデータフレームに、先ほどの危機遺産率を新たな列として加えます。

危機遺産率が0のものを取り除き、インデックスもリセットします。

`plotly`の`px.scatter_geo()`に引数`projection="orthographic"`を渡して地球儀を表示します。

どんなことが読み取れますか？

### 3. 横持ち・縦持ちへの変換

ここでは、データの各行に１つずつの情報を持つ縦持ちの状態からクロス集計表のような横持ちの状態への変換、またはその逆への変換をおこないます。

#### 3-1. 横持ちへの変換

縦軸を国、横軸をカテゴリとするクロス集計表を作ってみましょう。

まずは国、カテゴリでグループ化して件数を出力します。

横持ちになるよう、`unstack()`でカテゴリのそれぞれの値を列にします。  
また、該当する値がない場合は0とします。

これでクロス集計は完成です。

`groupby()`を使わず、`crosstab()`を使う方法でもやってみましょう。

さらに`pivot_table()`でも同じクロス集計を作成してみましょう。

数値計算でどんな関数を使うのか指定する必要があります。

最後に積み上げ横棒グラフでデータを多い順に表示してみましょう。

まず国ごとにカテゴリ数を合計して新たな列`Total`に格納後、`Total`列の値の降順に並べ替えます。

グラフを表示します。

どんなことが読み取れますか？

#### 3-2. 縦持ちへの変換

今度は手順が複雑になりますが、地域別のC1〜C6、N7〜N10の判断基準の合計数をレーダーチャートで表示してみます。

手順は以下になります。  
1.同じ地域での`id_no`の重複する行を除去。  
2.縦軸を地域、横軸を判断基準とするクロス集計のデータフレームを作成。  
3.地域のうち、複数の地域に跨るものを除去。  
4.データフレームを整形し、最終的に「地域」「判断基準」「合計数」の列となるようにする。  
5.`ploty`の`line_polar()`でレーダーチャートを表示する。  

まず重複の除去をおこないます。

次にクロス集計のデータフレームを作成します。

インデックスから複数の地域に跨るものを除去します。

判断基準をインデックスにした後、すべてのインデックスをリセットして縦持ちへの変換を完了します。

`plotly`の`line_ploar()`レーダーチャートを表示します。

どんなことが読み取れますか？

### 4. 世界遺産登録の変遷を地図上に表示

`plotly`のアニメーション機能を使って、世界遺産登録の変遷を地図上に表示してみましょう。

年が進むにつれて各国の総登録数が増えていき、国ごとの区画の色が濃くなるようにします。

まず最初に国名コードのデータを取得します。

地図上の国の区画とデータは3文字の国名コード（ISO 3166-1 alpha-3）で結びつけるのですが、世界遺産データの2文字の国名コードしかありませんので、データをウィキペディアから取得しロードします。

一度取得したデータは、何度も取得しないように`data/country_code.csv`保存するようにしましょう。


ウィキペディア ISO 3166-1  
https://ja.wikipedia.org/wiki/ISO_3166-1


取得したデータの中身を表示。

英語の国名を`country`、3文字のコードを`country_code`として、他の必要ない列は削除します。

次に地図用のデータフレームを用意します。  
以下の4つの属性を縦持ちしたデータフレームが必要です。

1. 国名コード  
世界遺産の登録があるすべての国の国名コード。

2. 国名  
世界遺産データにある英語名。
 
3. 年度  
世界全体で一番最初の世界遺産の登録年の前年から最新の登録の年度まで。  
すべての国が、すべての年度の行を持つ必要があります。

3. 登録数累計  
各国の各年度ごとの登録数累計。

作成手順は以下のようになります。

1. 世界遺産データを国、登録年でグループ化し、件数を集計した`Series`オブジェクトを作成。
2. 世界遺産の登録があるすべての国とすべての年度を掛け合わせたインデックスを持つ、値が空の`Series`オブジェクトを作成。
3. 2を1で上書きし、値が空の部分はすべて0で埋める。
4. 国、年度ごとに累計を計算し、新たな列として加える。
5. 国、年度のインデックスをリセットして列に変換し、データフレームにする。
6. 国名をキーに国名コードのデータフレームと結合する。

世界遺産データを国、登録年でグループ化し、件数を集計した`Series`オブジェクトを作成します。

世界遺産の登録があるすべての国とすべての年度を掛け合わせたインデックスを持つ、値が空の`Series`オブジェクトを作成します。

年度は世界全体で一番最初の世界遺産の登録年の前年から最新の登録の年度までです。

作成した空の`Series`オブジェクトを先ほどの件数のある`Series`オブジェクトで上書きし、値が空の部分はすべて0で埋めます。

上書きには`combine_first()`を使います。

国、年度ごとに累計を計算し、新たな列として加えます。

`Zimbabwe`（ジンバブエ）のデータで正しく累計が計算されているか確認します。

問題なければ国、年度のインデックスをリセットして列に変換し、データフレームにします。

各列名は`country`、`year`、`total`とします。

今度は国名をキーに国名コードのデータフレームと結合します。

国名コードがない行があるか確認し、あればその国名を取得します。

コードが見つからなかった国名と、正しいコードは以下になります。  
１つ１つ登録してしまいます。

- "Democratic People's Republic of Korea": PRK
- 'Democratic Republic of the Congo': COD
- 'Gambia (the)': GMB
- 'Jerusalem (Site proposed by Jordan)': JOR
- 'Palestine': PSE
- 'Republic of Korea': KOR
- 'United Republic of Tanzania': TZA
- 'Republic of Moldova': MDA

もう一度、コードがない国がないか確認します。

それでは`plotly`の`px.choropleth()`を利用して地図を表示します。

どんなことが読み取れますか？

### 5. 独自の分析目標を定義して分析してみましょう

〇〇〇〇〇〇〇〇〇〇を分析。